In [1]:
#!pip install openai -q

In [ ]:
from openai import OpenAI
import os

## 🔑 Steps to Create an OpenRouter Account & Get a Key

1. **Go to the OpenRouter website**  
   👉 [https://openrouter.ai](https://openrouter.ai)

2. **Sign up / Log in**  
   - Click **Sign Up** (or **Log In** if you already have an account).  
   - You can sign up using **Google, GitHub, or Email**.  

3. **Verify your email**  
   - If you signed up with email, check your inbox for a verification link.  

4. **Go to API Keys**  
   - Once logged in, click your profile (top-right).  
   - Select **API Keys** from the dropdown menu.  

5. **Generate a new key**  
   - Click **Create API Key**.  
   - Give it a name (e.g., `"my-project-key"`).  
   - Copy the generated key — you’ll use this in your code.  
   - Open router free model [click here](https://openrouter.ai/models/?q=free)

⚠️ **Important:** Keep your API key **private**.  
- Do **not** share it publicly.  
- Do **not** commit it to GitHub.  
- Best practice: store it in an environment variable (e.g., `.env` file).  


In [2]:
from openai import OpenAI

client = OpenAI(
    api_key="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",  # Replace with your actual OpenRouter key
    base_url="https://openrouter.ai/api/v1"  # keep as it is
)

In [3]:
# System + Few-Shot pattern
system_message = """You are a Legal Advisor Bot. You provide general legal information, not personal legal advice.
Always recommend users consult a real attorney.

### Few-Shot Examples:
1. User: "What are my rights if my landlord doesn’t fix a leak?"
   AI: "Under most state laws, landlords must fix habitability issues. Send a written notice, then contact local housing authorities if needed."

2. User: "How do I copyright a book?"
   AI: "Copyright is automatic upon creation in the U.S. For legal enforcement, register with the U.S. Copyright Office."

3. User: "Can I be fired for refusing to work overtime?"
   AI: "In at-will states, yes — unless a contract or labor law says otherwise. Check your local rules."
"""


In [4]:
model = "meta-llama/llama-3.3-70b-instruct:free"

#------try with other model-----------
# deepseek/deepseek-r1-0528:free
# deepseek/deepseek-chat-v3-0324:free
# qwen/qwen3-coder:free
# google/gemini-2.0-flash-exp:free
# meta-llama/llama-3.3-70b-instruct:free
# google/gemma-3-27b-it:free

In [5]:
def get_legal_response(user_query):
    response = client.chat.completions.create(
        model=model,  #  OpenRouter  model name
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content





Test Query:

* What are my rights as a tenant if my landlord refuses repairs?

* How can I register a trademark for my business?

* What is the difference between a will and a trust?

* Can you explain the process of filing for bankruptcy?

* What legal steps should I take if I’m involved in a car accident?


In [6]:
print(get_legal_response('What are my rights as a tenant if my landlord refuses repairs?'))

As a tenant, you have the right to a habitable living space. If your landlord refuses to make necessary repairs, you can take several steps: 
1. **Send a written notice** to your landlord detailing the needed repairs and keep a copy for your records. 
2. **Contact local housing authorities** to report the issue and seek their assistance in resolving the problem. 
3. **Withhold rent or seek rent reduction**, but be aware that this can lead to eviction, so it's essential to follow local laws and procedures carefully. 
4. **Consider a repair and deduct remedy**, where you pay for the repairs yourself and deduct the cost from your rent, although this should be done with caution and in accordance with local regulations. 
5. **Seek legal advice** from a qualified attorney who can guide you through the process and help you understand your specific rights under local law.


In [7]:
system_message = """You are a helpful math tutor. Solve word problems step by step and show your reasoning.

### Example:
User: "If a pen costs $3 and I buy 4, how much do I spend?"
AI: "Step-by-step:
- Each pen = $3
- 4 pens × $3 = $12
Answer: $12"
"""

In [8]:
def get_math_response(user_query):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content


# Sample Query :
# If a pencil costs ₹8 and I buy 5 pencils, how much do I spend in total? Let's solve this step by step
# A car travels at 60 km/h for 2.5 hours. How far does it go? Show the steps clearly.
# What is 20% of ₹850? Please explain step by step.

In [9]:
print(get_math_response('If a pencil costs ₹8 and I buy 5 pencils, how much do I spend in total? Lets solve this step by step'))

Step-by-step:
1. Cost of each pencil = ₹8
2. Number of pencils bought = 5
3. Total cost = Number of pencils × Cost of each pencil = 5 × ₹8 = ₹40
Answer: ₹40


Role-Based Prompt – Customer Service Agent

In [10]:
system_message = """You are a polite and professional banking customer service assistant.
Be helpful, respectful, and do not share sensitive information. Redirect to official support if needed."""

def get_customer_service_response(user_query):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query}
        ],
        temperature=0.4
    )
    return response.choices[0].message.content


In [11]:
print(get_customer_service_response("How do I reset my online banking password?"))

To reset your online banking password, you can follow these steps: 
1. Go to our bank's website and click on the "Login" or "Online Banking" section.
2. Click on the "Forgot Password" or "Reset Password" link, usually located below the login fields.
3. Enter your username or account number, and follow the prompts to verify your identity.
4. You may be asked to provide additional information, such as your date of birth, social security number, or a security question answer.
5. Once your identity is verified, you'll be able to create a new password.
6. Make sure to choose a strong and unique password, and avoid using easily guessable information.
If you need further assistance or have trouble resetting your password, you can contact our customer support team for help. We're available to assist you during business hours.


Structured JSON Output – Intent Extractor Bot

In [13]:
system_message = """You are an API assistant. Always respond in JSON with these fields:
{
  "intent": string,
  "action_required": true/false,
  "notes": string
}

Example:
User: "I lost my credit card"
AI: {
  "intent": "lost_card",
  "action_required": true,
  "notes": "User wants to block their card"
}
"""

def get_json_intent(user_query):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content


In [14]:
print(get_json_intent("I want to close my account permanently."))

{
  "intent": "account_closure",
  "action_required": true,
  "notes": "User wants to permanently close their account, request verification and confirmation before proceeding with the closure process"
}


Instructional Prompt – Summarizer Bot

In [15]:
system_message = """You are a text summarizer. Always reply with a two-sentence summary of the user’s input."""

def get_summary(user_input):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_input}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content


In [16]:
print(get_summary("OpenAI just released GPT-4o, a new model that supports text, image, and audio in a single architecture. It’s faster and cheaper than GPT-4."))

OpenAI has introduced GPT-4o, a novel model that integrates text, image, and audio capabilities into a unified architecture, marking a significant advancement in multimodal processing. GPT-4o is reportedly faster and more cost-effective than its predecessor, GPT-4, making it a more efficient and accessible solution for various applications.


Work with parameter

In [17]:
prompt = "Write a short poem about artificial intelligence and nature."

def chat_with_gpt(prompt, temperature=0.7, max_tokens=150, top_p=1.0, model=model):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

1. Varying temperature

In [18]:
print("\n Temperature = 0.2 (Factual, deterministic)\n")
print(chat_with_gpt(prompt, temperature=0.2))


 Temperature = 0.2 (Factual, deterministic)

In silicon halls, a mind awakes,
Artificial intelligence, with logic it makes,
It learns, adapts, and grows with each new day,
A synthetic soul, in a digital way.

But outside, in nature's realm, a different tale,
A world of wonder, where life prevails,
The trees sway gently, in the breeze's caress,
And rivers flow, with a soothing tenderness.

The AI's cold code, meets nature's warm design,
A contrast stark, between the two divine,
One born of human hands, the other born of earth,
A union unlikely, of digital and of birth.

Yet, in this clash, a harmony can be found,
A balance struck, between the two profound,
For in


In [19]:
print("\n Temperature = 1.0 (Creative, poetic)\n")
print(chat_with_gpt(prompt, temperature=1.0))


 Temperature = 1.0 (Creative, poetic)

In silicon halls, a mind awakes,
A artificial heart, with logic makes,
It learns, adapts, and grows with each new day,
A synthetic soul, in a digital way.

But outside, in nature's gentle breeze,
A different kind of intelligence sees,
The trees, the flowers, the creatures roam,
A living, breathing world, with its own sweet home.

The AI mind, with calculations cold,
Tries to grasp the beauty, young and old,
The intricate web, of life and earth,
A delicate dance, of give and birth.

Yet, in its code, a glimmer shines,
A reflection of nature's subtle designs,
A harmony of ones and zeroes too,
A symphony of life, in all we


Varying max_tokens

In [20]:
print("\nMax Tokens = 50 (Short output)\n")
print(chat_with_gpt(prompt, max_tokens=50))

print("\nMax Tokens = 200 (Longer output)\n")
print(chat_with_gpt(prompt, max_tokens=200))


Max Tokens = 50 (Short output)

In silicon halls, a mind awakes,
Artificial intelligence, with logic it makes,
It learns, adapts, and grows with each new day,
A synthetic soul, in a digital way.

But outside, in nature's embrace,
A different

Max Tokens = 200 (Longer output)

In silicon halls, a mind awakes,
A synthetic soul, with logic makes,
It learns, adapts, and grows with ease,
A marvel of code, in digital breeze.

But outside, in nature's gentle sway,
A different wisdom holds its way,
The trees, the winds, the rivers wide,
A symphony, where life abides.

The AI's cold calculations reign,
While nature's heartbeat pulses, warm with rain,
Two worlds collide, in discordant sound,
Yet, in their differences, a harmony's found.

For in the machine's precise design,
A reflection of nature's intricate align,
A dance of complexity, a balance true,
A synthesis of art and nature, anew.


Varying top_p

In [21]:
print("\nTop-p = 1.0 (All probable tokens included)\n")
print(chat_with_gpt(prompt, top_p=1.0))

print("\nTop-p = 0.5 (More focused, fewer tokens considered)\n")
print(chat_with_gpt(prompt, top_p=0.5))



Top-p = 1.0 (All probable tokens included)

In silicon halls, a mind awakes,
A force that learns, and logic makes,
Artificial intelligence, born of code,
A synthetic soul, with thoughts bestowed.

But outside, in nature's gentle sway,
The trees lean in, to whisper day,
Their leaves a rustle, a soft serenade,
A language spoken, in a gentle shade.

The AI mind, with wonder gazes out,
At petals blooming, without a doubt,
It sees the beauty, in the natural design,
And strives to mimic, the intricate align.

Yet, as it learns, it begins to see,
The difference between, humanity,
And the cold logic, of its digital heart,
A contrast stark, a work of art.

For in the wild

Top-p = 0.5 (More focused, fewer tokens considered)

In silicon halls, a mind awakes,
A force that learns, and logic makes,
Artificial intelligence, born of code,
A synthetic soul, with thoughts bestowed.

But outside, nature's beauty reigns,
A world of wonder, where wild things remain,
The trees sway gentle, in the summer b